In [88]:
import pandas as pd
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

import optuna
from boruta import BorutaPy as boruta
from sklearn.model_selection import StratifiedKFold
from optuna.visualization import plot_optimization_history, plot_param_importances


pd.set_option('display.max_columns', None)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daood\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\daood\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('sample30.csv', index_col=None)
data.shape

(30000, 15)

In [3]:
print(data['manufacturer'].nunique())
print(data['name'].nunique())
print(data['categories'].nunique())
print(data['brand'].nunique())

227
271
270
214


In [4]:
df = data[(data['reviews_username'].isnull()==False) & (data['user_sentiment'].isnull()==False)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29936 entries, 0 to 29999
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    29936 non-null  object
 1   brand                 29936 non-null  object
 2   categories            29936 non-null  object
 3   manufacturer          29795 non-null  object
 4   name                  29936 non-null  object
 5   reviews_date          29896 non-null  object
 6   reviews_didPurchase   15931 non-null  object
 7   reviews_doRecommend   27395 non-null  object
 8   reviews_rating        29936 non-null  int64 
 9   reviews_text          29936 non-null  object
 10  reviews_title         29747 non-null  object
 11  reviews_userCity      1900 non-null   object
 12  reviews_userProvince  166 non-null    object
 13  reviews_username      29936 non-null  object
 14  user_sentiment        29936 non-null  object
dtypes: int64(1), object(14)
memory usage

In [5]:
df['user_sentiment'].value_counts()

Positive    26579
Negative     3357
Name: user_sentiment, dtype: int64

In [6]:
df = df[['id', 'brand', 'categories', 'manufacturer', 'name', 'reviews_text', 'reviews_title', 'user_sentiment']]
df.shape

(29936, 8)

In [7]:
df.isnull().sum()

id                  0
brand               0
categories          0
manufacturer      141
name                0
reviews_text        0
reviews_title     189
user_sentiment      0
dtype: int64

In [8]:
df['reviews_title'] = df['reviews_title'].astype('O')
cnt = [i for i in df['reviews_title'].to_list() if isinstance(i, float)]
print(cnt)
cnt = [i for i in df['reviews_text'].to_list() if i.isnumeric()]
print(cnt)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[]


In [9]:
df['reviews_title'] = df['reviews_title'].fillna('NF')
df = df.reset_index(drop=True)

In [10]:
for i in range(len(df)):
  if df.loc[i,'reviews_title'] == 'NF':
    if df.loc[i, 'user_sentiment'] == 'Positive':
      df.loc[i,'reviews_title']='Good'
    if df.loc[i, 'user_sentiment'] == 'Negative':
      df.loc[i,'reviews_title']='Bad'

df['manufacturer'].fillna(df['brand'], inplace=True)

In [11]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

# add stemming and lemmatisation in the preprocess function
def preprocess(document):
    'changes document to lower case and removes stopwords'
    document = document.lower()
    words = word_tokenize(document)
    words = [word for word in words if word not in stopwords.words("english")]
    words = [stemmer.stem(word) for word in words]
    document = " ".join(words)
    return document

In [12]:
df['reviews_text'] = df['reviews_title']+' '+df['reviews_text']
df['reviews_text'][0]

"Just Awesome i love this album. it's very good. more to the hip hop side than her current pop sound.. SO HYPE! i listen to this everyday at the gym! i give it 5star rating all the way. her metaphors are just crazy."

In [13]:
df['proc_reviews_text'] = df['reviews_text'].apply(preprocess)
df.shape

(29936, 9)

In [14]:
# df.to_csv('final_data.csv', index=None)

In [64]:
df = pd.read_csv('final_data.csv', index_col=None)
df.head(2)

,id,brand,categories,manufacturer,name,reviews_text,reviews_title,user_sentiment,proc_reviews_text
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),Just Awesome i love this album. it's very good...,Just Awesome,Positive,awesom love album . 's good . hip hop side cur...
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,Good Good flavor. This review was collected as...,Good,Positive,good good flavor . review collect part promot .


In [65]:
df['user_sentiment'] = df['user_sentiment'].map({'Positive':1, 'Negative':0})
df['user_sentiment'].value_counts()

1    26579
0     3357
Name: user_sentiment, dtype: int64

In [66]:
xtrain, xtest, ytrain ,ytest = train_test_split(df['proc_reviews_text'], df['user_sentiment'], test_size=0.3)

In [67]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_model = tfidf_vectorizer.fit(xtrain)

xtrain_tfi = tfidf_model.transform(xtrain)
xtest_tfi  = tfidf_model.transform(xtest)

xdf = pd.DataFrame(xtrain_tfi.toarray(), columns=tfidf_vectorizer.get_feature_names())
print(xdf.shape)
xdf.head(2)

(20955, 11401)


,00,000,0000,007,079,10,100,1000,1007,100free,100percent,10280,10281,103,1040,1080,1080p,10ish,10min,10mo,10x,10year,10yr,11,110,113,11320,11321,1138,11438,1183,12,1200,120mg,122016,123,12443842,127201,12lb,12month,12th,12year,13,130,1300,135lb,1375,13yr,14,140,140lb,149,15,150,1500,15oz,16,160,1600,17,17lb,18,180,1800,184,1850,185lb,186,188,18month,18yr,19,190,1917,1921,1931,1935,1941,1949,195,1950,1954,1960,1961,1964,1965,1970,1971,1975,1980,19800702984,1982,1983,1984,1986,1988,1990,1994,1995,1996,1997,1998,1999,19g,19mo,19th,1g,1hour,1hr,1keywordsvegeta,1st,1t,1tsp,1x,1year,20,200,2000,2001,2002,2003,2005,2007,2009,2010,20114,2012,2013,2014,2015,2016,2017,205lb,2062351337,20min,20th,20x,21,2175046722,219,21st,22,2200,23,2300,237ml,2392280232,23rd,24,240,24x32,25,250,250lb,250w,2532674594,25483701,26,27,270,27yr,28,285ml,29,29product,2b,2d,2disc,2hand,2in1,2nd,2u,2x,2yr,30,300,3000,300lb,30yr,31,310,32,33,332,3375509742,34,35,36,360,367,37,370,372,37499,38,39,3b,3c,3d,3d4k,3di,3g,3lb,3oz,3pk,3rd,3s,3small,3x,3yo,3yr,40,400,4048197640,40mg,40min,41,42,44,45,46,47,48,48hour,48hr,49,4a,4b,4c,4h,4k,4kreview,4ktv,4th,4x,50,500,500g,5030,50inch,50s,51,52,52yr,53,53yr,54,55,55min,56,57,5742870423,58,59,5auido,5extra,5hdr,5ish,5min,5movi,5oz,5p,5plu,5star,5t,5th,5x11,5yr,60,600,6000,60th,61,62,63,6389,64,65,65gm,66,6623443094,67,6787249844,67oz,68,6am,6th,70,700,71,72,7250,73,74,75,76,7month,7th,80,800,81,83,84,85,8574178605,86,86project,88,89,8hr,8pm,90,9003058363,90th,95,96,960,96yr,97,98,99,9a,9lb,9yr,a1,aaa,aaahhhhh,aaammmaaaz,aalot,aanyhwer,aaron,ab,abandon,abc,abcess,abd,abdomin,abil,abl,aboard,abomin,abort,abosolut,abou,above,abras,abroad,abruptli,absenc,absob,absolut,absolutey,absolutley,absolutli,absorb,absout,abt,abund,abus,abysm,ac,academ,academi,accent,accentu,accept,access,accessori,accessout,accid,accident,acclim,accommod,accomod,accompani,accomplish,accord,accordingli,account,accredit,acct,acction,accumul,accur,accus,accustom,accutan,acdc,acess,aceton,ach,achiev,achievedi,acid,ack,acknowledg,acknowleg,acn,aco,acoss,acquaint,acquir,acquisit,acroo,across,acryl,acrylic,act,actaulli,acting,actingand,action,actioni,actionmovi,activ,activist,acton,actor,actorfrom,actors,actress,actual,actuali,acut,acuum,ad,adam,adapt,add,addict,addit,address,adequ,adequate,adher,adhes,adhesvi,adjac,adjus,adjust,administr,admint,admir,admiss,admit,admittedli,admitttedli,ador,adore,adriana,adul,adult,adults,advanc,advantag,advent,adventur,adventure,advers,advertis,advertised,advi,advic,advice,advis,ae,aerial,aerosol,aesthet,aesthetician,af,afew,affair,affect,affix,afford,affraid,affrid,affter,afghanistan,aficionado,afield,aforement,aforesaid,afraid,afresh,africa,african,afro,aft,after,afterlif,aftermath,afternoon,aftershav,aftertast,afterthought,afterward,afterword,ag,agai,again,againt,age,agedefi,agedfi,agenc,agenda,agent,aggrav,aggress,agit,agnsti,ago,agon,agre,agreat,agree,agreement,agress,ah,ahead,ahhh,ahhhhhhh,ahhhhhhhh,ahmaz,ahol,ahoy,ai,aid,ail,ailment,aim,aint,aio,air,airborn,airbrush,aircon,aircraft,aired,airi,airlin,airplan,airport,airspun,airtight,airway,aisl,aittl,ajust,aka,akin,akira,akso,al,ala,alan,alarm,alarms,alaska,albacor,albeit,alberto,albertson,alblum,album,alcohol,alday,aldean,aldeanlov,alden,aldo,ale,alec,alergi,alert,alevi,alex,alexand,alfredo,alga,alic,alien,align,alik,alittl,aliv,alkyl,all,allegr,allen,allerg,allergen,allergi,allevi,alley,alli,alll,allll,allot,allov,allow,allstar,alltim,allur,allway,almond,almost,alo,alon,along,alongsid,alot,alpha,alreadi,alright,alsean,also,alter,altern,althoug,although,altitud,alto,altogeth,alwasi,alway,alwi,alwya,am,amandaf,amax,amaz,amazebal,amazed,amazing,amazinggg,amazingggg,amazinggggg,amazingli,amazingy,amazon,amazong,amber,ambi,amc,amen,amercan,america,american,ami,amid,aminah,amlway,ammonia,ammonium,amond,among,amongst,amount,amp,ampl,amplifi,amsingli,amt,amus,amz,an,analyz,anaphylact,anchor,ancient,and,andcharact,andean,anderson,andi,andonopolu,andrew,android,andveri,anecd

In [68]:
bow_vectorizer = CountVectorizer(stop_words='english', max_features=8000)
bow_model = bow_vectorizer.fit(xtrain)

xtrain_bow = bow_model.transform(xtrain)
xtest_bow  = bow_model.transform(xtest)

xdf_bow = pd.DataFrame(xtrain_bow.toarray(), columns=bow_vectorizer.get_feature_names())
print(xdf_bow.shape)
xdf_bow.head(2)

(20955, 8000)


00  000  0000  007  079  10  100  1000  1007  100free  100percent  10280  \
0   0    0     0    0    0   0    0     0     0        0           0      0   
1   0    0     0    0    0   0    0     0     0        0           0      0   

   10281  103  1040  1080  1080p  10ish  10min  10mo  10x  10year  10yr  11  \
0      0    0     0     0      0      0      0     0    0       0     0   0   
1      0    0     0     0      0      0      0     0    0       0     0   0   

   110  113  11320  11321  1138  11438  1183  12  1200  120mg  122016  123  \
0    0    0      0      0     0      0     0   0     0      0       0    0   
1    0    0      0      0     0      0     0   0     0      0       0    0   

   12443842  127201  12lb  12month  12th  12year  13  130  1300  135lb  1375  \
0         0       0     0        0     0       0   0    0     0      0     0   
1         0       0     0        0     0       0   0    0     0      0     0   

   13yr  14  140  140lb  149  15  150  1500  15oz  16  160  1600  17  17lb  \
0     0   0    0      0    0   0    0     0     0   0    0     0   0     0   
1     0   0    0      0    0   0    0     0     0   0    0     0   0     0   

   18  180  1800  184  1850  185lb  186  188  18month  18yr  19  190  1917  \
0   0    0     0    0     0      0    0    0        0     0   0    0     0   
1   0    0     0    0     0      0    0    0        0     0   0    0     0   

   1921  1931  1935  1941  1949  195  1950  1954  1960  1961  1964  1965  \
0     0     0     0     0     0    0     0     0     0     0     0     0   
1     0     0     0     0     0    0     0     0     0     0     0     0   

   1970  1971  1975  1980  19800702984  1982  1983  1984  1986  1988  1990  \
0     0     0     0     0            0     0     0     0     0     0     0   
1     0     0     0     0            0     0     0     0     0     0     0   

   1994  1995  1996  1997  1998  1999  19g  19mo  19th  1g  1hour  1hr  \
0     0     0     0     0     0     0    0     0     0   0      0    0   
1     0     0     0     0     0     0    0     0     0   0      0    0   

   1keywordsvegeta  1st  1t  1tsp  1x  1year  20  200  2000  2001  2002  2003  \
0                0    0   0     0   0      0   0    0     0     0     0     0   
1                0    0   0     0   0      0   0    0     0     0     0     0   

   2005  2007  2009  2010  20114  2012  2013  2014  2015  2016  2017  205lb  \
0     0     0     0     0      0     0     0     0     0     0     0      0   
1     0     0     0     0      0     0     0     0     0     0     0      0   

   2062351337  20min  20th  20x  21  2175046722  219  21st  22  2200  23  \
0           0      0     0    0   0           0    0     0   0     0   0   
1           0      0     0    0   0           0    0     0   0     0   0   

   2300  237ml  2392280232  23rd  24  240  24x32  25  250  250lb  250w  \
0     0      0           0     0   0    0      0   0    0      0     0   
1     0      0           0     0   0    0      0   0    0      0     0   

   2532674594  25483701  26  27  270  27yr  28  285ml  29  29product  2b  2d  \
0           0         0   0   0    0     0   0      0   0          0   0   0   
1           0         0   0   0    0     0   0      0   0          0   0   0   

   2disc  2hand  2in1  2nd  2u  2x  2yr  30  300  3000  300lb  30yr  31  310  \
0      0      0     0    0   0   0    0   0    0     0      0     0   0    0   
1      0      0     0    0   0   0    0   0    0     0      0     0   0    0   

   32  33  332  3375509742  34  35  36  360  367  37  370  372  37499  38  39  \
0   0   0    0           0   0   0   0    0    0   0    0    0      0   0   0   
1   0   0    0           0   0   0   0    0    0   0    0    0      0   0   0   

   3b  3d  3di  3oz  3rd  3x  40  400  40min  42  44  45  47  48  48hour  \
0   0   0    0    0    0   0   0    0      0   0   0   0   0   0       0   
1   0   0    0    0    0   0   0    0      0   0   0   0   0   0       0   

   48

In [76]:
list(bow_vectorizer.vocabulary_.items())[:10]

[('use', 7310),
 ('daili', 1887),
 ('favorit', 2860),
 ('shampoo', 5905),
 ('fine', 2944),
 ('hair', 3522),
 ('perfect', 4945),
 ('smell', 6151),
 ('good', 3397),
 ('price', 5170)]

In [69]:
xtrain_tfi.shape, xtest_tfi.shape, xtrain_bow.shape, xtest_bow.shape

((20955, 11401), (8981, 11401), (20955, 8000), (8981, 8000))

### Logistic Regression Model

In [70]:
logreg1 = LogisticRegression(penalty="l2", random_state=42, C=3.5)
logreg1.fit(xtrain_tfi, ytrain)

D:\Programs\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=3.5, random_state=42)

In [71]:
pred_train = logreg1.predict(xtrain_tfi)
pred_test  = logreg1.predict(xtest_tfi)
print("Train accuracy: ", accuracy_score(pred_train, ytrain))
print("Test accuracy: ", accuracy_score(pred_test, ytest))
print("Train AUC: ", roc_auc_score(pred_train, ytrain))
print("Test AUC: ", roc_auc_score(pred_test, ytest))

Train accuracy:  0.9427821522309712
Test accuracy:  0.9114797906691905
Train AUC:  0.9271923380378272
Test AUC:  0.837871933044517


In [72]:
logreg2 = LogisticRegression(penalty="l2", random_state=42, C=3.5)
logreg2.fit(xtrain_bow, ytrain)

D:\Programs\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=3.5, random_state=42)

In [73]:
pred_train2 = logreg2.predict(xtrain_bow)
pred_test2  = logreg2.predict(xtest_bow)
print("Train accuracy (BOW): ", accuracy_score(pred_train2, ytrain))
print("Test accuracy (BOW): ", accuracy_score(pred_test2, ytest))
print("Train AUC (BOW): ", roc_auc_score(pred_train, ytrain))
print("Test AUC (BOW): ", roc_auc_score(pred_test, ytest))

Train accuracy (BOW):  0.9654974946313529
Test accuracy (BOW):  0.9143747912259214
Train AUC (BOW):  0.9271923380378272
Test AUC (BOW):  0.837871933044517


### Naive Bayes Model

In [77]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb.fit(xtrain_bow, ytrain)

BernoulliNB()

In [78]:
pred_train = bnb.predict(xtrain_bow)
pred_test  = bnb.predict(xtest_bow)
print("Train accuracy (BOW): ", accuracy_score(pred_train2, ytrain))
print("Test accuracy (BOW): ", accuracy_score(pred_test2, ytest))
print("Train AUC (BOW): ", roc_auc_score(pred_train, ytrain))
print("Test AUC (BOW): ", roc_auc_score(pred_test, ytest))

Train accuracy (BOW):  0.9654974946313529
Test accuracy (BOW):  0.9143747912259214
Train AUC (BOW):  0.7066835936544683
Test AUC (BOW):  0.6551597019964699


### XGBoost Classifier

In [79]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(xtrain_bow, ytrain)

D:\Programs\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:34:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [80]:
pred_train = xgb.predict(xtrain_bow)
pred_test  = xgb.predict(xtest_bow)
print("Train accuracy (BOW): ", accuracy_score(pred_train2, ytrain))
print("Test accuracy (BOW): ", accuracy_score(pred_test2, ytest))
print("Train AUC (BOW): ", roc_auc_score(pred_train, ytrain))
print("Test AUC (BOW): ", roc_auc_score(pred_test, ytest))

Train accuracy (BOW):  0.9654974946313529
Test accuracy (BOW):  0.9143747912259214
Train AUC (BOW):  0.9145715482924784
Test AUC (BOW):  0.839555551090303


In [87]:
params = {
    'min_child_weight': [1, 5],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [3, 4, 5]
}

AttributeError: 'XGBClassifier' object has no attribute 'best_params_'

### Random Forest Model

In [82]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(xtrain_bow, ytrain)

RandomForestClassifier()

In [83]:
pred_train = rf.predict(xtrain_bow)
pred_test  = rf.predict(xtest_bow)
print("Train accuracy (BOW): ", accuracy_score(pred_train2, ytrain))
print("Test accuracy (BOW): ", accuracy_score(pred_test2, ytest))
print("Train AUC (BOW): ", roc_auc_score(pred_train, ytrain))
print("Test AUC (BOW): ", roc_auc_score(pred_test, ytest))

Train accuracy (BOW):  0.9654974946313529
Test accuracy (BOW):  0.9143747912259214
Train AUC (BOW):  0.9997590143912857
Test AUC (BOW):  0.8658990278608474
